In [ ]:
import pandas as pd
import numpy as np
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

train_data = pd.read_csv('train.csv')
sample_submission = pd.read_csv('sample_submission.csv')


df_train = train_data.copy()
df_train.drop('id', axis=1,inplace=True)
df_test = pd.read_csv('test.csv')
df_test.drop('id', axis=1,inplace=True)
df_origin = pd.read_csv('horse.csv')


df_train_total = pd.concat([df_train, df_origin], ignore_index=True)
df_train_total.drop_duplicates(inplace=True)

df_total = pd.concat([df_train_total, df_test], ignore_index=True)

target = 'outcome'

# df_train.duplicated().sum()
df_total.duplicated().sum()

df_total[target] = df_total[target].map({'died':0,'euthanized':1,'lived':2})



def preprocessing(df, le_cols, ohe_cols):

    # Label Encoding for binary cols
    le = LabelEncoder()
    for col in le_cols:
        df[col] = le.fit_transform(df[col])

    # OneHot Encoding for category cols
    df = pd.get_dummies(df, columns = ohe_cols)

    df["pain"] = df["pain"].replace('slight', 'moderate')
    df["peristalsis"] = df["peristalsis"].replace('distend_small', 'normal')
    df["rectal_exam_feces"] = df["rectal_exam_feces"].replace('serosanguious', 'absent')
    df["nasogastric_reflux"] = df["nasogastric_reflux"].replace('slight', 'none')

    df["temp_of_extremities"] = df["temp_of_extremities"].fillna("normal").map({'cold': 0, 'cool': 1, 'normal': 2, 'warm': 3})
    df["peripheral_pulse"] = df["peripheral_pulse"].fillna("normal").map({'absent': 0, 'reduced': 1, 'normal': 2, 'increased': 3})
    df["capillary_refill_time"] = df["capillary_refill_time"].fillna("3").map({'less_3_sec': 0, '3': 1, 'more_3_sec': 2})
    df["pain"] = df["pain"].fillna("depressed").map({'alert': 0, 'depressed': 1, 'moderate': 2, 'mild_pain': 3, 'severe_pain': 4, 'extreme_pain': 5})
    df["peristalsis"] = df["peristalsis"].fillna("hypomotile").map({'hypermotile': 0, 'normal': 1, 'hypomotile': 2, 'absent': 3})
    df["abdominal_distention"] = df["abdominal_distention"].fillna("none").map({'none': 0, 'slight': 1, 'moderate': 2, 'severe': 3})
    df["nasogastric_tube"] = df["nasogastric_tube"].fillna("none").map({'none': 0, 'slight': 1, 'significant': 2})
    df["nasogastric_reflux"] = df["nasogastric_reflux"].fillna("none").map({'less_1_liter': 0, 'none': 1, 'more_1_liter': 2})
    df["rectal_exam_feces"] = df["rectal_exam_feces"].fillna("absent").map({'absent': 0, 'decreased': 1, 'normal': 2, 'increased': 3})
    df["abdomen"] = df["abdomen"].fillna("distend_small").map({'normal': 0, 'other': 1, 'firm': 2,'distend_small': 3, 'distend_large': 4})
    df["abdomo_appearance"] = df["abdomo_appearance"].fillna("serosanguious").map({'clear': 0, 'cloudy': 1, 'serosanguious': 2})

    # Imputer
    cols_with_nan = df.drop(target,axis=1).columns[df.drop(target,axis=1).isna().any()].tolist()
    for feature in cols_with_nan:
        df[feature].fillna(df[feature].mode()[0], inplace=True)

    return df




df_total = preprocessing(df_total, le_cols = ["surgery", "age", "surgical_lesion", "cp_data"], ohe_cols = ["mucous_membrane"])

df_train = df_total[df_total[target].notna()]
df_test = df_total[df_total[target].isna()]
df_test.drop(target,axis=1,inplace=True)


full_features = df_test.columns.tolist()
bin_features = df_test.select_dtypes('bool').columns

df_train[bin_features] = df_train[bin_features].astype('int64')
df_test[bin_features] = df_test[bin_features].astype('int64')


best_features = full_features.copy()
len(best_features)
best_features.remove('lesion_2')
best_features.remove('lesion_3')
len(best_features)

X_opt = df_train[best_features]
Y_opt = df_train[target]
df_pred = df_test[best_features]


# -------------------- LightGBM Hyperparameter Tuning -------------------- #
param_grid_lgbm = {
    'n_estimators': [100, 200, 300],
    'max_depth': [4, 6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'min_child_samples': [10, 20, 30],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [0, 0.1, 0.5]
}

lgbm_model = LGBMClassifier(random_state=42)

# Use GridSearchCV to search for the best parameters
grid_search_lgbm = GridSearchCV(estimator=lgbm_model, param_grid=param_grid_lgbm,
                                cv=3, n_jobs=-1, verbose=2, scoring='accuracy')

# Fit the model
grid_search_lgbm.fit(X_opt, Y_opt)

# Retrieve the best model
best_lgbm_model = grid_search_lgbm.best_estimator_

# Output best parameters and accuracy
print(f"Best LightGBM Parameters: {grid_search_lgbm.best_params_}")



Fitting 3 folds for each of 8748 candidates, totalling 26244 fits


<ipython-input-2-bed3fdec96bf>:87: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[feature].fillna(df[feature].mode()[0], inplace=True)
<ipython-input-2-bed3fdec96bf>:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop(target,axis=1,inplace=True)
<ipython-input-2-bed3fdec96bf>:104: SettingWithCopyWarning: 
A value is trying to be s

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000172 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 657
[LightGBM] [Info] Number of data points in the train set: 1531, number of used features: 30
[LightGBM] [Info] Start training from score -1.147468
[LightGBM] [Info] Start training from score -1.653504
[LightGBM] [Info] Start training from score -0.710940
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further split

In [ ]:
# -------------------- Generate Test Predictions -------------------- #
# Generate predictions for the test dataset using the best LightGBM model
test_predictions = best_lgbm_model.predict(df_pred)

# Prepare the submission file
hist_opt_submission = pd.DataFrame({'id': sample_submission['id'], 'outcome': test_predictions})
hist_opt_submission['outcome'] = hist_opt_submission['outcome'].map({0: 'died', 1: 'euthanized', 2: 'lived'})
hist_opt_submission.to_csv('LGBM_Tuned_Submission.csv', index=False)
print("Submission file for LightGBM created successfully.")